In [1]:
import numpy as np
import sys
sys.path.append("../src")
from gPCS import gPCS
import matplotlib.pyplot as plt
import healpy as hp
from astropy.io import fits

NSIDE = 512

In [2]:
TS_star = 36
# pixels candidates without filtering
print(len(gPCS.get_firing_pixels(TS_star, filter=False))) 
# pixels candidates with filtering of pixels from the simulated 4FGL catalog
print(len(gPCS.get_firing_pixels(TS_star, filter=True, conservative=False))) 
# pixels candidates with conservative filtering of pixels from the simulated 4FGL catalog padded by 1 pixel
print(len(gPCS.get_firing_pixels(TS_star, filter=True, conservative=True)))
# pixels candidates with conservative filtering of pixels from the simulated 4FGL catalog padded by 1 pixels, 
# with further removal of pixels within 0.5 deg from ANY resolved 4FGL source (regardless of the TS value of the source in the 1-10 GeV band)
print(len(gPCS.get_firing_pixels(TS_star, filter=True, conservative=True, deg=0.5)))

9604
2719
1240
147


In [3]:
alpha=0.01
TS_star = gPCS.get_TS_from_QF(0.50, alpha=alpha) # given alpha and QF, we identify the TS value
firing_pixels = gPCS.get_firing_pixels(TS_star, filter=False) # we get the pixel candidates
TS_ranking = gPCS.TS_map_Fermi[firing_pixels] # we get the TS values for the pixel candidates

In [4]:
# we can obtain the galactic coordinates of a pixel using the healpy pix2ang function
lon, lat = hp.pix2ang(NSIDE, firing_pixels, lonlat=True) # lon, lat in degrees

In [5]:
# given a ranking of TS values, we can compute the associated mean QF and standard deviation
# this gives us an information about the QF range that we can expect for a given pixel
mean_QF, std_QF = gPCS.get_QF_ranges_from_TS(TS_ranking, alpha=alpha, batches=100, batch_size=3000)

In [6]:
# create a fits file 
gPCS.export_fits_table("firing_pixels.fits", 0.5, alpha=[0.01, 0.05, 0.1], overwrite=True)
# if we want, we can also apply filtering to the pixel candidates, passign extra keyword arguments
gPCS.export_fits_table("firing_pixels_filtered.fits", 0.5, alpha=[0.01, 0.05, 0.1], overwrite=True, filter=True, conservative=True)

'firing_pixels_filtered.fits'

In [7]:
# read the fits file
with fits.open("firing_pixels.fits") as f:
    display(f.info())
    display(f[1].header) # type: ignore
    data_001 = f[1].data  # type: ignore
    data_005 = f[2].data  # type: ignore
    data_01  = f[3].data  # type: ignore

print("data for alpha=0.01")
display(data_001)

Filename: firing_pixels.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  ALPHA=0.01    1 BinTableHDU     19   9957R x 5C   [K, E, E, E, E]   
  2  ALPHA=0.05    1 BinTableHDU     19   9500R x 5C   [K, E, E, E, E]   
  3  ALPHA=0.1     1 BinTableHDU     19   8681R x 5C   [K, E, E, E, E]   


None

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   24 / length of dimension 1                          
NAXIS2  =                 9957 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    5 / number of table fields                         
TTYPE1  = 'pixel   '                                                            
TFORM1  = 'K       '                                                            
TTYPE2  = 'TS      '                                                            
TFORM2  = 'E       '                                                            
TTYPE3  = 'QF_best '        

data for alpha=0.01


FITS_rec([( 146881, 6.8197925e+05, 1.        , 1.        , 1.        ),
          (2545130, 6.4414856e+05, 1.        , 1.        , 1.        ),
          (2543081, 3.3422578e+05, 1.        , 1.        , 1.        ),
          ...,
          (2907254, 3.3943626e+01, 0.50394815, 0.49927253, 0.5106475 ),
          (2427848, 3.3915901e+01, 0.50070864, 0.49590784, 0.5073455 ),
          ( 387044, 3.3909271e+01, 0.50050616, 0.4957333 , 0.50716674)],
         dtype=(numpy.record, [('pixel', '>i8'), ('TS', '>f4'), ('QF_best', '>f4'), ('QF_min', '>f4'), ('QF_max', '>f4')]))